In [45]:
import pandas as pd
import numpy as np
from feature_engineering import *
from sklearn.model_selection import train_test_split
from modeling import train_logistic_regression, train_xgboost, predict_model, save_submission
from metrics_k import score


In [46]:
df = pd.read_csv('dsb-24-german-credit/german_credit_train.csv')

In [47]:
df = engineer_features(df)

In [48]:
df

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,CreditHistoryRisk,ExistingSavingsNumeric,EmploymentDurationYears,loan_to_income_proxy,debt_to_savings,loan_amount_x_credit_risk,employment_stability,residence_to_employment_ratio,dependents_per_age,prior_default_flag
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,3,300,0.5,629.664568,6.296666,0,0.193548,5.999880,0.031250,0
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,3,50,2.5,230.998845,9.239998,0,1.666656,0.799997,0.027027,0
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,0,50,2.5,124.999375,4.999999,750,1.999984,1.199995,0.035714,1
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,3,50,8.0,1230.995897,73.859985,0,3.428557,0.250000,0.031250,0
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,0,750,8.0,2078.326406,8.313333,18705,3.428557,0.375000,0.017544,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,greater_200,27,credits_paid_to_date,furniture,4650,less_100,1_to_4,3,male,none,...,3,50,2.5,1549.994833,92.999981,0,1.111106,1.599994,0.025000,0
3995,0_to_200,11,prior_payments_delayed,furniture,250,greater_1000,4_to_7,3,male,none,...,0,1500,5.5,83.333056,0.166667,750,5.999935,0.545454,0.031250,1
3996,no_checking,32,outstanding_credit,appliances,6536,unknown,greater_7,5,male,co-applicant,...,1,<NA>,8.0,1307.197386,<NA>,13072,2.999989,0.624999,0.037037,0
3997,0_to_200,38,outstanding_credit,other,1597,500_to_1000,greater_7,3,female,co-applicant,...,1,750,8.0,532.331559,2.129333,3194,2.526308,0.375000,0.037037,0


In [49]:
X = df.drop(columns=['Risk'])
y = df['Risk']

X = pd.get_dummies(X, drop_first=True)
y= y.map({'No Risk': 0, 'Risk': 1})

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [50]:
model = train_xgboost(X_train, y_train)

/Users/marcosalerno/German_Credit/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [16:35:58] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [52]:
y_train_pred = predict_model(model, X_train)
y_train_pred_labels = pd.Series(y_train_pred).map({0: 'No Risk', 1: 'Risk'})
y_train_labels = y_train.map({0: 'No Risk', 1: 'Risk'})

df_train_true = pd.DataFrame({'Risk': y_train_labels, 'LoanAmount': df.loc[y_train.index, 'LoanAmount']})
df_train_pred = pd.DataFrame({'Risk': y_train_pred_labels})

train_cost = score(df_train_true, df_train_pred, row_id_column_name=None)
print(f"Train cost: {train_cost}")


y_val_pred = predict_model(model, X_val)
y_val_pred_labels = pd.Series(y_val_pred).map({0: 'No Risk', 1: 'Risk'})
y_val_labels = y_val.map({0: 'No Risk', 1: 'Risk'})

df_val_true = pd.DataFrame({'Risk': y_val_labels, 'LoanAmount': df.loc[y_val.index, 'LoanAmount']})
df_val_pred = pd.DataFrame({'Risk': y_val_pred_labels})

val_cost = score(df_val_true, df_val_pred, row_id_column_name=None)
print(f"Validation cost: {val_cost}")



Train cost: -50.83704978439815
Validation cost: -30.229011834319536


In [53]:
df_test = pd.read_csv('dsb-24-german-credit/german_credit_test.csv')
df_test = engineer_features(df_test)
X_test = pd.get_dummies(df_test, drop_first=True)

# Assicurati che le colonne di test matchino quelle di train:
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

test_predictions = predict_model(model, X_test)
test_predictions_labels = pd.Series(test_predictions).map({0: 'No Risk', 1: 'Risk'})

save_submission(test_predictions_labels, output_path='submission.csv')

Submission file saved to submission.csv
